In [1]:
import yfinance as yf
import numpy as np

class DCF:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def calculate_fcf(self):
        cashflow = self.stock.cashflow
        if 'Free Cash Flow' in cashflow.index:
            fcf = cashflow.loc['Free Cash Flow']
        else:
            print("Free Cash Flow data is not available.")
            fcf = None
        return fcf
    
    def calculate_wacc(self,ticker):
        balance_sheet = self.stock.balance_sheet
        income_statement = self.stock.financials
        ebit=income_statement.loc['EBIT']
        interest_expense = income_statement.loc['Interest Expense']
        tax_rate = 0.125
        risk_free_rate = 0.02
        market_preimum = 0.05
        beta=ticker.info['beta']
        cost_of_equity=risk_free_rate+beta*market_preimum
        cost_of_debt=(interest_expense)*(1-tax_rate)
        wacc=(cost_of_equity+cost_of_debt)*(1+tax_rate)
        return wacc.dropna()
    
    def terminal_value(self):
        fcf = self.calculate_fcf()
        wacc = self.calculate_wacc()
        growth_rate=0.04
        terminal_value=fcf*(1+growth_rate)/(wacc-growth_rate)
        terminal_value=terminal_value.dropna()
        return terminal_value[::-1]

    def calculate_dcf(self):
        fcf = self.calculate_fcf()  # List of free cash flows
        wacc = self.calculate_wacc()  # List of weighted average cost of capital for each period
        terminal_value = self.terminal_value()  # Terminal value at the end of projection

        if len(fcf) != len(wacc) + 1:
            print("Error: The length of FCF should be one more than WACC for terminal value calculation.")
            return []

        dcf = []  # Initialize the list to store discounted cash flows
    
        # Calculate discounted FCF for each year except the terminal value
        for i in range(len(wacc)):
            discounted_fcf = fcf[i] / (1 + wacc[i]) ** (i + 1)
            dcf.append(discounted_fcf)
            # Debugging: Print each discounted cash flow
            print(f"Year {i+1}: FCF = {fcf[i]}, WACC = {wacc[i]}, Discounted FCF = {discounted_fcf}")

        # Add the terminal value, discounted using the last WACC
        terminal_value_discounted = (fcf[-1] + terminal_value) / (1 + wacc[-1]) ** len(fcf)
        dcf.append(terminal_value_discounted)

        # Debugging: Print the final discounted terminal value
        print("Discounted Terminal Value:", terminal_value_discounted)
        
        print("Final DCF:", dcf)  # Debugging: Print the final DCF list
        return dcf


        
    
    
    def get_outstanding_shares(self):
        try:
            shares = self.stock.info['sharesOutstanding']
        except KeyError as e:
            print(f"KeyError: {e}")
            shares = None
        return shares
    
    def calculate_projected_share_price(self, dcf_value):
        shares_outstanding = self.get_outstanding_shares()
        if dcf_value is not None and shares_outstanding is not None:
            projected_share_price = dcf_value / shares_outstanding
        else:
            print("Cannot calculate projected share price due to missing data.")
            projected_share_price = None
        return projected_share_price

# Example usage
dcf_model = DCF('AAPL')  # Replace 'AAPL' with the desired ticker symbol
dcf_value = dcf_model.calculate_dcf()


Year 1: FCF = 99584000000.0, WACC = 3871546875.09, Discounted FCF = 25.722018404326565
Year 2: FCF = 111443000000.0, WACC = 2885203125.09, Discounted FCF = 1.3387516110184354e-08
Year 3: FCF = 92953000000.0, WACC = 2603671875.09, Discounted FCF = 5.2662885936972126e-18
Year 4: FCF = 73365000000.0, WACC = 2828109375.09, Discounted FCF = 1.1468433888228896e-27
Discounted Terminal Value: 2023-09-30    NaN
2022-09-30    NaN
2021-09-30    NaN
2020-09-30    NaN
Freq: -1YE-SEP, dtype: object
Final DCF: [25.722018404326565, 1.3387516110184354e-08, 5.2662885936972126e-18, 1.1468433888228896e-27, 2023-09-30    NaN
2022-09-30    NaN
2021-09-30    NaN
2020-09-30    NaN
Freq: -1YE-SEP, dtype: object]


/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_80254/3664131697.py:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  discounted_fcf = fcf[i] / (1 + wacc[i]) ** (i + 1)
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_80254/3664131697.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Year {i+1}: FCF = {fcf[i]}, WACC = {wacc[i]}, Discounted FCF = {discounted_fcf}")
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_80254/3664131697.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels

In [13]:
import yfinance as yf
import numpy as np

class DCF:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def calculate_fcf(self):
        cashflow = self.stock.cashflow
        if 'Free Cash Flow' in cashflow.index:
            fcf = cashflow.loc['Free Cash Flow']
        else:
            print("Free Cash Flow data is not available.")
            fcf = None
        return fcf
    
    def calculate_wacc(self):
        balance_sheet = self.stock.balance_sheet
        income_statement = self.stock.financials
        try:
            ebit = income_statement.loc['EBIT']
            interest_expense = income_statement.loc['Interest Expense']
        except KeyError:
            print("EBIT or Interest Expense data is not available.")
            return None

        tax_rate = 0.125  # Assume a tax rate, can be customized per company
        risk_free_rate = 0.02  # Example: US 10-year treasury yield
        market_premium = 0.05  # Historical average market premium
        beta = 1.2  # Example beta value for the company

        # Cost of Equity (CAPM)
        cost_of_equity = risk_free_rate + beta * market_premium

        # Cost of Debt
        try:
            cost_of_debt = (interest_expense * (1 - tax_rate)).mean()
        except:
            cost_of_debt = None
            print("Cost of debt calculation failed, missing data.")

        # Assume weights (debt/equity ratio) for simplicity
        equity_weight = 0.7
        debt_weight = 0.3

        if cost_of_debt is not None:
            wacc = (cost_of_equity * equity_weight) + (cost_of_debt * debt_weight)
        else:
            wacc = cost_of_equity * equity_weight  # Use equity WACC if debt info is unavailable
        
        return wacc
    
    def terminal_value(self, fcf):
        wacc = self.calculate_wacc()
        if wacc is None:
            return None
        growth_rate = 0.04  # Constant terminal growth rate, example value

        if wacc <= growth_rate:
            print("Error: WACC should be greater than the growth rate.")
            return None

        # Last year FCF is used for terminal value calculation
        terminal_value = fcf[-1] * (1 + growth_rate) / (wacc - growth_rate)
        return terminal_value

    def calculate_dcf(self):
        fcf = self.calculate_fcf()  # List of free cash flows
        if fcf is None or fcf.empty:
            print("FCF data is missing. Unable to calculate DCF.")
            return None

        wacc = self.calculate_wacc()  # Weighted average cost of capital
        if wacc is None:
            print("WACC calculation failed. Unable to calculate DCF.")
            return None

        terminal_value = self.terminal_value(fcf)  # Terminal value at the end of projection
        if terminal_value is None:
            print("Terminal value calculation failed.")
            return None

        dcf = []  # Initialize the list to store discounted cash flows
        n_years = len(fcf)

        # Calculate discounted FCF for each year except the terminal value
        for i in range(n_years):
            discounted_fcf = fcf[i] / (1 + wacc) ** (i + 1)
            dcf.append(discounted_fcf)

        # Add the terminal value, discounted using the last WACC
        terminal_value_discounted = terminal_value / (1 + wacc) ** n_years
        dcf.append(terminal_value_discounted)

        # Debugging: Print the final DCF list
        print("Final DCF with discounted terminal value:", dcf)
        return np.sum(dcf)  # Return the sum of discounted cash flows
    
    def get_outstanding_shares(self):
        try:
            shares = self.stock.info['sharesOutstanding']
        except KeyError as e:
            print(f"KeyError: {e}")
            shares = None
        return shares
    
    def calculate_projected_share_price(self):
        dcf_value = self.calculate_dcf()
        shares_outstanding = self.get_outstanding_shares()
        
        if dcf_value is not None and shares_outstanding is not None:
            projected_share_price = dcf_value / shares_outstanding
        else:
            print("Cannot calculate projected share price due to missing data.")
            projected_share_price = None
        return projected_share_price


# Example usage
dcf_model = DCF('AAPL')  # Replace 'AAPL' with the desired ticker symbol
projected_share_price = dcf_model.calculate_projected_share_price()
print("Projected Share Price:", projected_share_price)


Final DCF with discounted terminal value: [np.float64(122.55455291700842), np.float64(1.6878449991077323e-07), np.float64(1.7325391653631015e-16), np.float64(1.6828618114427363e-25), np.float64(nan), np.float64(nan)]
Projected Share Price: nan


/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_80254/3052830545.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  terminal_value = fcf[-1] * (1 + growth_rate) / (wacc - growth_rate)
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_80254/3052830545.py:89: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  discounted_fcf = fcf[i] / (1 + wacc) ** (i + 1)


In [12]:
dcf_model.calculate_dcf()

Final DCF with discounted terminal value: [np.float64(122.55455291700842), np.float64(1.6878449991077323e-07), np.float64(1.7325391653631015e-16), np.float64(1.6828618114427363e-25), np.float64(nan), np.float64(nan)]


/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_80254/3052830545.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  terminal_value = fcf[-1] * (1 + growth_rate) / (wacc - growth_rate)
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_80254/3052830545.py:89: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  discounted_fcf = fcf[i] / (1 + wacc) ** (i + 1)


np.float64(nan)

# Best Version

In [27]:
import yfinance as yf
import numpy as np

class DCF:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def calculate_fcf(self):
        cashflow = self.stock.cashflow
        if 'Free Cash Flow' in cashflow.index:
            fcf = cashflow.loc['Free Cash Flow'].dropna()  # Ensure no NaN values
            print("Free Cash Flow (FCF) Data:", fcf)
        else:
            print("Free Cash Flow data is not available.")
            fcf = None
        return fcf
    
    def calculate_wacc(self):
        ticker = yf.Ticker(self.ticker)
        income_statement = self.stock.financials
        
        try:
            ebit = income_statement.loc['EBIT'].iloc[0]
            interest_expense = income_statement.loc['Interest Expense'].iloc[0]
        except (KeyError, IndexError):
            print("EBIT or Interest Expense data is not available.")
            return None

        tax_rate = 0.125  # Example tax rate
        risk_free_rate = 0.02  # Example: US 10-year treasury yield
        market_premium = 0.05  # Historical average market premium
        beta =  ticker.info['beta'] # Example beta value for the company

        # Cost of Equity (CAPM)
        cost_of_equity = risk_free_rate + beta * market_premium

        # Cost of Debt (after tax)
        try:
            cost_of_debt = (interest_expense * (1 - tax_rate)) / ebit  # Adjust for taxes
        except ZeroDivisionError:
            print("Error calculating cost of debt.")
            cost_of_debt = None

        # Assume weights (debt/equity ratio) for simplicity
        equity_weight = 0.7
        debt_weight = 0.3

        if cost_of_debt is not None:
            wacc = (cost_of_equity * equity_weight) + (cost_of_debt * debt_weight)
        else:
            wacc = cost_of_equity * equity_weight  # Use equity WACC if debt info is unavailable
        
        print("Calculated WACC:", wacc)
        return wacc
    
    def terminal_value(self, fcf):
        wacc = self.calculate_wacc()
        if wacc is None:
            return None
        growth_rate = 0.04  # Constant terminal growth rate, example value

        if wacc <= growth_rate:
            print("Error: WACC should be greater than the growth rate.")
            return None

        # Last year FCF is used for terminal value calculation
        try:
            terminal_value = fcf.iloc[-1] * (1 + growth_rate) / (wacc - growth_rate)
        except (IndexError, KeyError, ZeroDivisionError):
            print("Error calculating terminal value due to missing FCF or invalid WACC.")
            terminal_value = None
        return terminal_value

    def calculate_dcf(self):
        fcf = self.calculate_fcf()
        if fcf is None or fcf.empty:
            print("FCF data is missing. Unable to calculate DCF.")
            return None

        wacc = self.calculate_wacc()
        if wacc is None:
            print("WACC calculation failed. Unable to calculate DCF.")
            return None

        terminal_value = self.terminal_value(fcf)
        if terminal_value is None:
            print("Terminal value calculation failed.")
            return None

        dcf = []  # Initialize the list to store discounted cash flows
        n_years = len(fcf)

        # Calculate discounted FCF for each year except the terminal value
        for i in range(n_years):
            try:
                discounted_fcf = fcf.iloc[i] / (1 + wacc) ** (i + 1)  # Correctly access FCF with iloc
                dcf.append(discounted_fcf)
            except (IndexError, KeyError, ZeroDivisionError):
                print(f"Error calculating discounted FCF for year {i+1}.")
                dcf.append(0)  # Append 0 if there's an error

        # Add the terminal value, discounted using the last WACC
        try:
            terminal_value_discounted = terminal_value / (1 + wacc) ** n_years
            dcf.append(terminal_value_discounted)
        except ZeroDivisionError:
            print("Error discounting terminal value.")
            dcf.append(0)  # Append 0 if there's an error

        # Debugging: Print the final DCF list
        print("Final DCF with discounted terminal value:", dcf)
        return np.nansum(dcf)  # Return the sum of discounted cash flows, ignore NaN values
    
    def get_outstanding_shares(self):
        try:
            shares = self.stock.info['sharesOutstanding']
        except KeyError as e:
            print(f"KeyError: {e}")
            shares = None
        return shares
    
    def calculate_projected_share_price(self):
        dcf_value = self.calculate_dcf()
        shares_outstanding = self.get_outstanding_shares()
        
        if dcf_value is not None and shares_outstanding is not None:
            projected_share_price = dcf_value / shares_outstanding
        else:
            print("Cannot calculate projected share price due to missing data.")
            projected_share_price = None
        return projected_share_price


# Example usage
dcf_model = DCF('MSFT')  # Replace 'AAPL' with the desired ticker symbol
projected_share_price = dcf_model.calculate_projected_share_price()
print("Projected Share Price:", projected_share_price)


Free Cash Flow (FCF) Data: 2024-06-30    74071000000.0
2023-06-30    59475000000.0
2022-06-30    65149000000.0
2021-06-30    56118000000.0
Name: Free Cash Flow, dtype: object
Calculated WACC: 0.05231830548581132
Calculated WACC: 0.05231830548581132
Final DCF with discounted terminal value: [70388398276.3224, 53708150885.83336, 55907024454.31924, 45762915388.83706, 3863634658128.136]
Projected Share Price: 550.1653721162022


In [21]:
import yfinance as yf

# Define the stock symbol
symbol = 'AAPL'

# Create a Ticker object
ticker = yf.Ticker(symbol)



# Access the beta value
beta = ticker.info['beta']

print(f"The beta value for {symbol} is {beta}")

The beta value for AAPL is 1.24


In [35]:
import yfinance as yf

# Define the stock symbol
symbol = 'AAPL'

# Create a Ticker object
ticker = yf.Ticker(symbol)
debt=np.array(ticker.balance_sheet.loc['Total Debt'])
equity=np.array(ticker.balance_sheet.loc['Total Equity Gross Minority Interest'])


In [37]:
debt/(equity+debt)

array([0.6660181861175004, 0.7233336245304447, 0.6839368374646815,
       0.6517426459222778, nan], dtype=object)

In [50]:
result=(equity/(equity+debt))[0]
float(result)

0.33398181388249965

In [54]:
import yfinance as yf
import numpy as np

class DCF:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def calculate_fcf(self):
        cashflow = self.stock.cashflow
        if 'Free Cash Flow' in cashflow.index:
            fcf = cashflow.loc['Free Cash Flow'].dropna()  # Ensure no NaN values
            print("Free Cash Flow (FCF) Data:", fcf)
        else:
            print("Free Cash Flow data is not available.")
            fcf = None
        return fcf
    
    def calculate_wacc(self):
        ticker = yf.Ticker(self.ticker)
        income_statement = self.stock.financials
        
        try:
            ebit = income_statement.loc['EBIT'].iloc[0]
            interest_expense = income_statement.loc['Interest Expense'].iloc[0]
        except (KeyError, IndexError):
            print("EBIT or Interest Expense data is not available.")
            return None

        tax_rate = 0.125  # Example tax rate
        risk_free_rate = 0.02  # Example: US 10-year treasury yield
        market_premium = 0.04  # Historical average market premium
        beta =  ticker.info['beta'] # Example beta value for the company

        # Cost of Equity (CAPM)
        cost_of_equity = risk_free_rate + beta * market_premium

        # Cost of Debt (after tax)
        try:
            cost_of_debt = (interest_expense * (1 - tax_rate)) / ebit  # Adjust for taxes
        except ZeroDivisionError:
            print("Error calculating cost of debt.")
            cost_of_debt = None

        # Assume weights (debt/equity ratio) for simplicity
        debt=np.array(ticker.balance_sheet.loc['Total Debt'])
        equity=np.array(ticker.balance_sheet.loc['Total Equity Gross Minority Interest'])
        equity_weight = float((equity/(equity+debt))[0])
        debt_weight = float((debt/(equity+debt))[0])

        if cost_of_debt is not None:
            wacc = (cost_of_equity * equity_weight) + (cost_of_debt * debt_weight)
        else:
            wacc = cost_of_equity * equity_weight  # Use equity WACC if debt info is unavailable
        
        print("Calculated WACC:", wacc)
        return wacc
    
    def terminal_value(self, fcf):
        wacc = self.calculate_wacc()
        if wacc is None:
            return None
        growth_rate = 0.04  # Constant terminal growth rate, example value

        if wacc <= growth_rate:
            print("Error: WACC should be greater than the growth rate.")
            return None

        # Last year FCF is used for terminal value calculation
        try:
            terminal_value = fcf.iloc[-1] * (1 + growth_rate) / (wacc - growth_rate)
        except (IndexError, KeyError, ZeroDivisionError):
            print("Error calculating terminal value due to missing FCF or invalid WACC.")
            terminal_value = None
        return terminal_value

    def calculate_dcf(self):
        fcf = self.calculate_fcf()
        if fcf is None or fcf.empty:
            print("FCF data is missing. Unable to calculate DCF.")
            return None

        wacc = self.calculate_wacc()
        if wacc is None:
            print("WACC calculation failed. Unable to calculate DCF.")
            return None

        terminal_value = self.terminal_value(fcf)
        if terminal_value is None:
            print("Terminal value calculation failed.")
            return None

        dcf = []  # Initialize the list to store discounted cash flows
        n_years = len(fcf)

        # Calculate discounted FCF for each year except the terminal value
        for i in range(n_years):
            try:
                discounted_fcf = fcf.iloc[i] / (1 + wacc) ** (i + 1)  # Correctly access FCF with iloc
                dcf.append(discounted_fcf)
            except (IndexError, KeyError, ZeroDivisionError):
                print(f"Error calculating discounted FCF for year {i+1}.")
                dcf.append(0)  # Append 0 if there's an error

        # Add the terminal value, discounted using the last WACC
        try:
            terminal_value_discounted = terminal_value / (1 + wacc) ** n_years
            dcf.append(terminal_value_discounted)
        except ZeroDivisionError:
            print("Error discounting terminal value.")
            dcf.append(0)  # Append 0 if there's an error

        # Debugging: Print the final DCF list
        print("Final DCF with discounted terminal value:", dcf)
        return np.nansum(dcf)  # Return the sum of discounted cash flows, ignore NaN values
    
    def get_outstanding_shares(self):
        try:
            shares = self.stock.info['sharesOutstanding']
        except KeyError as e:
            print(f"KeyError: {e}")
            shares = None
        return shares
    
    def calculate_projected_share_price(self):
        dcf_value = self.calculate_dcf()
        shares_outstanding = self.get_outstanding_shares()
        
        if dcf_value is not None and shares_outstanding is not None:
            projected_share_price = dcf_value / shares_outstanding
        else:
            print("Cannot calculate projected share price due to missing data.")
            projected_share_price = None
        return projected_share_price


# Example usage
dcf_model = DCF('MSFT')  # Replace 'AAPL' with the desired ticker symbol
projected_share_price = dcf_model.calculate_projected_share_price()
print("Projected Share Price:", projected_share_price)


Free Cash Flow (FCF) Data: 2024-06-30    74071000000.0
2023-06-30    59475000000.0
2022-06-30    65149000000.0
2021-06-30    56118000000.0
Name: Free Cash Flow, dtype: object
Calculated WACC: 0.049310267223136556
Calculated WACC: 0.049310267223136556
Final DCF with discounted terminal value: [70590179390.90532, 54016520570.1423, 56389206967.366745, 46289926944.16046, 5170799383963.158]
Projected Share Price: 726.2284759389131
